In [96]:
import requests
import bs4
import csv
from selenium import webdriver
from time import sleep
import urllib3
from lxml import etree
from selenium.webdriver.common.action_chains import ActionChains

In [97]:
#職業详情
def get_one_page(url):
    http = urllib3.PoolManager(timeout=2)
    urllib3.disable_warnings()
    response = http.request('get', url)
    html = response.data
    selector = etree.HTML(html)
    detail=selector.xpath("/html/body/div/div[3]/div/div[2]/ul/li")
    info=[]
    job_title=selector.xpath("//li/div/div/h3/a/div[@class='job-title']/text()")
    yearly_salary=selector.xpath("//li/div/div/h3/a/span/text()")
    place=selector.xpath("//li/div/div[1]/p/text()[1]")
    seniority=selector.xpath("//li/div/div[1]/p/text()[2]")
    degree=selector.xpath("//li/div/div[1]/p/text()[3]")
    description=selector.xpath("//li/div/div[1]/h3/a/div[2]/div[2]/div[2]/text()")
    for i in range(0,len(detail)):
        info.append([job_title[i],yearly_salary[i],place[i],seniority[i],degree[i]])
    return info

def get_one_page_description(url):
    driver = webdriver.Firefox()
    driver.get(url)
    driver.set_window_size(10000,5080)#设置窗口宽度，以免浮出的窗口超出范围
    chain = ActionChains(driver)#联动鼠标
    descriptions=[]
    for i in range(1,31):
        stop=driver.find_element_by_xpath( "/html/body/div/div[3]/div/div[2]/ul/li["+ str(i) + "]/div/div[1]/h3/a/div[1]" )
        chain.move_to_element(stop).perform()#停留鼠标
        sleep(1)
        description=driver.find_element_by_xpath("//html/body/div/div[3]/div/div[2]/ul/li["+ str(i) + "]/div/div[1]/h3/a/div[2]/div[2]/div[2]").text
        descriptions.append(description)
        driver.refresh
    return descriptions

def get_all_pages(url_list):
    all_info=[]
    for url in url_list:
        info=get_one_page(url)
        for i in info:
            all_info.append(i)
    return all_info 

def get_all_descriptions(url_list):
    all_descriptions=[]
    for url in url_list:
        des=get_one_page_description(url)
        for i in des:
            all_descriptions.append(i)
    return all_descriptions

In [98]:
base_url_list=['https://www.zhipin.com/i100003-c100010000/?page=',
               'https://www.zhipin.com/i100101-c100010000/?page=',
               'https://www.zhipin.com/c100010000-p140101/?page=',
               'https://www.zhipin.com/i100012-c100010000/?page=',
               'https://www.zhipin.com/i100009-c100010000/?page=',
               'https://www.zhipin.com/i100011-c100010000/?page=',
              'https://www.zhipin.com/c100010000/e_102/?page=']
url_list=[]
for i in range(1,11):
    for base_url in base_url_list:
        url=base_url+str(i)
        url_list.append(url)

In [99]:
all_jobs=get_all_pages(url_list)

In [100]:
with open('jobs_info_media.csv','w',newline='') as g:
    writer = csv.writer(g)
    header = ['job_title','yearly_salary','place','seniority','degree']
    writer.writerow(header)
    writer.writerows(all_jobs)
len(all_jobs)

2059

In [62]:
import pandas as pd
df=pd.read_csv('jobs_info_media.csv',encoding ='gbk') 
df=df.drop_duplicates()